In [1]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Minimizing a Finite State Machine

In this notebook we show how a <span style="font-variant:small-caps;">Dfa</span> can be minimized by identifying states the are equivalent.

## Some Type Definitions

* `Char` is an alias for `str`.
* `State` is the abstract type of the states that make up the deterministic <span style="font-variant:small-caps;">Fsm</span>.

In [2]:
type Char = string;
type State = number;

`TransRel` is the type that describes the transition function $\delta$.

In [3]:
type TransRel = Map<State, Map<Char, State>>;

`DFA` is the type that describes the deterministic <span style="font-variant:small-caps;">Fsm</span>.

In [4]:
interface DFA {
  Q: Set<State>;                 // set of states
  Sigma: Set<Char>;              // alphabet
  delta: TransRel;               // transition function
  q0: State;                     // start state
  F: Set<State>;                 // accepting states
}

`Pair` is a pair of states.

In [5]:
type Pair = [State, State];

The states of the minimized <span style="font-variant:small-caps;">Fsm</span> will be  sets of states of the original <span style="font-variant:small-caps;">Fsm</span>.  These states will contain those states that are *equivalent*.

In [6]:
type SetState = ReadonlySet<State>;

In [7]:
type TransRelSet = Map<string, SetState>;

In [8]:
interface MinDFA {
  Q: Set<SetState>;
  Sigma: Set<Char>;
  delta: TransRelSet;
  q0: SetState;
  F: Set<SetState>;
}

The function `arb(M)` takes a non-empty set `M` as its argument and returns an *arbitrary* element from this set.  The set `M` is not changed.  The function `arb`also works if `M` is a `frozenset`. 

In order to type-check this function we define an abstract type `S` that serves as a *type parameter* and  denotes the type of the elements of the set `M`.

In [9]:
 function arb<S>(M: Set<S> | ReadonlySet<S>): S {
  for (const x of M) return x;
  throw new Error("Error: arb called with empty set!");
}

The function `cart_prod(A, B)` computes the *Cartesian product* $A \times B$ of the sets $A$ and $B$ where $A \times B$ is defined as follows:
$$ A \times B := \{ (x, y) \mid x \in A \wedge y \in B \}. $$
In order to denote the type of the second set, we define the abstract type `T`.

In [10]:
function cartProd<S, T>(A: Set<S>, B: Set<T>): Set<[S, T]> {
  const result = new Set<[S, T]>();
  for (const a of A) {
    for (const b of B) {
      result.add([a, b]);
    }
  }
  return result;
}

In [11]:
// Beispielaufruf für cartProd
const A = new Set([1, 2]);
const B = new Set(['a', 'b', 'c']);

const result = cartProd(A, B);

// Ergebnis schön ausgeben:
console.log(new Set([...result].map(([x, y]) => `(${x}, '${y}')`)));

Set(6) {
  "(1, 'a')",
  "(1, 'b')",
  "(1, 'c')",
  "(2, 'a')",
  "(2, 'b')",
  "(2, 'c')"
}


The function `separate` takes four arguments:
- `Pairs`  is a set of pairs of states from some given <span style="font-variant:small-caps;">Dfa</span> $F$.

   If $(p_1, p_2) \in \texttt{Pairs}$, then $p_1$ and $p_2$ are known to be *separable*.
   
   Two states $p_1$ and $p_2$ are *separable* if there is a string $w$ such that processing $w$ in state
   $p_1$ leads to an accepting state, while processing $w$ in state $p_2$ leads to a state that is not 
   accepting, or vice versa.
- `States` is the set of all states of the <span style="font-variant:small-caps;">Fsm</span> $F$,
- `Σ` is the alphabet of the <span style="font-variant:small-caps;">Dfa</span> $F$.
- `𝛿` is the transition function of the <span style="font-variant:small-caps;">Dfa</span>

The function `separate(Pairs, States, Σ, 𝛿)` computes the set of pairs of states $(q_1, q_2)$ that are *separable* because there is some character $c \in \Sigma$ such that 
$$\delta(q_1,c) = p_1, \quad \textrm{but} \quad \delta(q_2,c) = p_2 $$
and $(p_1, p_2)$ is already known to be *separable* because $(p_1, p_2) \in \textrm{Pairs}$.

In [12]:
function separate(
  Pairs: Set<Pair>,
  States: Set<State>,
  Sigma: Set<Char>,
  delta: TransRel
): Set<Pair> {
  const Result = new Set<Pair>();

  // Prüft, ob ein Paar (p1, p2) in Pairs existiert
  const hasPair = (p1: State | undefined, p2: State | undefined): boolean => {
    if (p1 === undefined || p2 === undefined) return false;
    for (const [x, y] of Pairs) {
      if (x === p1 && y === p2) return true;
    }
    return false;
  };

  for (const q1 of States) {
    for (const q2 of States) {
      for (const c of Sigma) {
        const next1 = delta.get(q1)?.get(c);
        const next2 = delta.get(q2)?.get(c);
        if (hasPair(next1, next2)) {
          Result.add([q1, q2]);
        }
      }
    }
  }

  return Result;
}

Given a state `p` and a `Partition` of the set of all states, the function `find_equivalence_class(p, Partition)` returns the equivalence class of `p`, i.e. it returns the set of states from `Partition` that contains the state `x`. 

In [13]:
function findEquivalenceClass(
  p: State,
  Partition: Set<SetState>
): SetState {
  const classes = new Set<SetState>();

  for (const C of Partition) {
    if (C.has(p)) {
      classes.add(C);
    }
  }

  // Falls mehrere oder keine Klassen gefunden werden:
  if (classes.size === 0) {
    throw new Error(`State ${p} not found in any equivalence class!`);
  }

  // Gib eine beliebige (die einzige) Klasse zurück
  return arb(classes);
}

The function `reachable(q0, Σ, 𝛿)` takes three arguments:
* `q0` is the start state of an Fsm,
* `Σ`  is the alphabet.
* `𝛿`  is the transition function.  The transition function is assumed to be *complete*. `𝛿` is represented as a dictionary.   

It returns the set of all states that can be reached from the start state `q0` by reading strings of characters from `Σ`.

The result is computed via a *fixed point* computation.

In [14]:
function reachable(
  q0: State,
  Sigma: Set<Char>,
  delta: TransRel
): Set<State> {
  const Result = new Set<State>([q0]);

  while (true) {
    const NewStates = new Set<State>();

    for (const p of Result) {
      const transitions = delta.get(p);
      if (!transitions) continue;
      for (const c of Sigma) {
        const next = transitions.get(c);
        if (next !== undefined) {
          NewStates.add(next);
        }
      }
    }

    // Prüfen, ob NewStates schon in Result enthalten ist
    let allKnown = true;
    for (const s of NewStates) {
      if (!Result.has(s)) {
        allKnown = false;
        Result.add(s);
      }
    }

    if (allKnown) return Result; // Fixpunkt erreicht
  }
}

The function `all_separable(Q, A, Σ, 𝛿)` takes four arguments:
* `Q` is the set of states of the Fsm.
* `A` is the set of all accepting states,
* `Σ`  is the alphabet.
* `𝛿` is the transition function.  

  `𝛿` is represented as a dictionary.   

The function computes the set of all Pairs `(p, q)` such that `p` and `q` are separable, i.e. all pairs such that
$$ \exists s \in \Sigma^*: \bigl(\delta^*(p, s) \in A \wedge \delta^*(q,s) \not\in A\bigr) \vee 
                           \bigl(\delta^*(p, s) \not\in A \wedge \delta^*(q,s) \in A\bigr). 
$$
The result is computed via a *fixed point* computation:
1. Initially, all accepting states are separated from the non-accepting states.
2. If $p_1$ and $p_2$ are separable and $q_1$ and $q_2$ are two states such that there is a character 
   $c \in \Sigma$ such that
   $$ \delta(q_1, c) = p_1 \quad \mbox{and} \quad \delta(q_2, c) = p_2, $$
   then $q_1$ and $q_2$ are separable.

In [15]:
function allSeparable(
  Q: Set<State>,
  A: Set<State>,
  Sigma: Set<Char>,
  delta: TransRel
): Set<Pair> {
  // Anfang: akzeptierende vs. nicht-akzeptierende Zustände
  const nonAccepting = new Set([...Q].filter((q) => !A.has(q)));

  // cartProd(Q − A, A) ∪ cartProd(A, Q − A)
  let Separable = new Set<Pair>([
    ...cartProd(nonAccepting, A),
    ...cartProd(A, nonAccepting),
  ]);

  while (true) {
    const NewPairs = separate(Separable, Q, Sigma, delta);

    // Prüfen, ob NewPairs eine echte Erweiterung ist
    let isSubset = true;
    for (const pair of NewPairs) {
      if (![...Separable].some(([x, y]) => x === pair[0] && y === pair[1])) {
        isSubset = false;
        break;
      }
    }

    if (isSubset) {
      return Separable; // Fixpunkt erreicht
    }

    // Vereinigung Separable ∪ NewPairs
    for (const pair of NewPairs) {
      Separable.add(pair);
    }
  }
}

The function `minimize(A)` takes a <span style="font-variant:small-caps;">Dfa</span> `F` as its input.
Here `F` is a 5-tuple of the form
$$ F = (Q, \Sigma, \delta, q_0, A) $$
The algorithm performs the following steps:
1. All unreachable states are eliminated.
2. All accepting states are separated form all non-accepting states.
3. States are separated as long as possible.
   Two states $p_1$ and $p_2$ are separable if there is a string 
   $w \in \Sigma^*$ such that 
   $$ \Bigl(\delta(p_1,w) \in A \;\wedge\; \delta(p_2,w) \not\in A\Bigr) \quad \vee \quad 
      \Bigl(\delta(p_1,w) \not\in A \;\wedge\; \delta(p_2,w) \in A\Bigr)
   $$
4. States that are not separable are *equivalent* and are therefore identified and grouped
   in equivalence classes.  The states in an equivalence class are then identified.

In [16]:
function minimize(F: DFA): MinDFA {
  let { Q, Sigma, delta, q0, F: Accepting } = F;

  // 1️⃣ Eliminate unreachable states
  Q = reachable(q0, Sigma, delta);

  // 2️⃣ Compute separable pairs
  const Separable = allSeparable(Q, Accepting, Sigma, delta);

  // 3️⃣ Compute equivalent pairs (Q×Q \ Separable)
  const Equivalent = new Set<Pair>();
  for (const [p, q] of cartProd(Q, Q)) {
    if (![...Separable].some(([x, y]) => x === p && y === q)) {
      Equivalent.add([p, q]);
    }
  }

  // 4️⃣ Build equivalence classes
  const EquivClasses = new Set<SetState>();
  for (const q of Q) {
    const cls = new Set<State>();
    for (const p of Q) {
      if ([...Equivalent].some(([x, y]) => x === p && y === q)) {
        cls.add(p);
      }
    }
    EquivClasses.add(cls);
  }

  // 5️⃣ Determine new start state
  const newQ0 = arb(new Set([...EquivClasses].filter((C) => C.has(q0))));

  // 6️⃣ Determine new accepting states
  const newAccept = new Set<SetState>(
    [...EquivClasses].filter((C) => {
      const rep = arb(C);
      return Accepting.has(rep);
    })
  );

  // 7️⃣ Build new transition function
  const newDelta: TransRelSet = new Map();

  for (const q of Q) {
    const classOfQ = findEquivalenceClass(q, EquivClasses);
    for (const c of Sigma) {
      const next = delta.get(q)?.get(c);
      if (next !== undefined) {
        const classOfP = findEquivalenceClass(next, EquivClasses);
        newDelta.set(JSON.stringify([Array.from(classOfQ), c]), classOfP);
      } else {
        newDelta.set(JSON.stringify([Array.from(classOfQ), c]), new Set());
      }
    }
  }

  // 8️⃣ Return minimized DFA
  return {
    Q: EquivClasses,
    Sigma,
    delta: newDelta,
    q0: newQ0,
    F: newAccept,
  };
}